In [1]:
from datasets import load_dataset
from transformers import GPT2Tokenizer, GPT2Config, GPT2LMHeadModel
from torch.optim import AdamW
from torch.utils.data import DataLoader
from transformers import pipeline

/home/alysson/workspace/Criando-um-LLM-modelo-de-linguagem-de-grande-escala-do-zero-com-Transformers/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load dataset
dataset = load_dataset("wikitext", "wikitext-103-v1")
train_data = dataset['train']['text']

In [4]:
# Create tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token  # Set padding token

In [5]:
# Encode data
train_encodings = tokenizer(train_data, return_tensors='pt', max_length=0.1, truncation=True, padding="max_length")

In [ ]:
# Create model config and model
config = GPT2Config.from_pretrained('gpt2')
model = GPT2LMHeadModel(config)

In [ ]:
# Create optimizer and data loader
optimizer = AdamW(model.parameters(), lr=5e-5)
train_loader = DataLoader(train_encodings, batch_size=8, shuffle=True)

In [ ]:
# Train model
model.train()
for epoch in range(5):
    for batch in train_loader:
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = input_ids.clone()
        labels[labels == tokenizer.pad_token_id] = -100  # Ignore padding in loss calculation
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    print(f"Epoch {epoch}: Loss {loss.item()}")

In [ ]:
# Save model
model.save_pretrained('./meuModeloGPT2')

In [ ]:
# Create generator pipeline
generator = pipeline('text-generation', model='./meuModeloGPT2', tokenizer='gpt2')

In [ ]:
# Generate text
prompt = "Qual é o significado da vida, do universo e tudo mais?"
response = generator(prompt, max_length=50, num_return_sequences=1)
print(response[0]['generated_text'])